***SETUP:***

In [ ]:
pip install python-terrier==0.10.0 nltk scikit-learn lightgbm

In [107]:
##UNCOMMENT IF WORKING ON COLAB!

# from google.colab import drive
# drive.mount('/content/drive')

In [108]:
import pandas as pd
import json
import pyterrier as pt
import random

***DATASETS CREATION AND PREPROCESSING:***

In [109]:
#UNCOMMENT ONE OF THE FOLLOWING LINES TO USE EITHER THE TVR DATASET OR THE QVH DATASET

dataset_choice = "TVR"
# dataset_choice = "QVH"

In [110]:
# Paths to JSONL files. 
if dataset_choice == "TVR": 
    jsonl_train_path = 'text_data/tvr_train_release.jsonl'
    jsonl_val_path = 'text_data/tvr_val_release.jsonl'
    subs_path = 'text_data/tvqa_preprocessed_subtitles.jsonl'

elif dataset_choice == "QVH":
    jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
    subs_path = "text_data_QVH/subs_train.jsonl"

##UNCOMMENT IF WORKING ON COLAB!
# if dataset_choice == "TVR": 
#     jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
#     jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
#     subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#     jsonl_train_path = "/content/drive/MyDrive/IR/text_data_QVH/highlight_train_release.jsonl"
#     subs_path = "/content/drive/MyDrive/IR/text_data_QVH/subs_train.jsonl"

In [111]:
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
if dataset_choice == "TVR":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            subtitles_dict[sub_data['vid_name']] = sub_data['sub']
elif dataset_choice == "QVH":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            triple = sub_data['vid'].split("_")
            name = triple[0:-2]
            #turn the list name into a string
            name = "".join(name)
            if name not in subtitles_dict:
                subtitles_dict[name] = [(float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query'])]
            else:
                subtitles_dict[name].append((float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query']))

In [112]:
# Function to find matching subtitles in TVR case
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]) or (subtitle['start'] <= ts_range[0] and subtitle['end'] >= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [113]:
def parse_jsonl_TVR(jsonl_path, split_type):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)
            # drop non text-based queries
            if data['type'] not in ['t']: #add vt to include video-text queries
                continue

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            if matching_subs == []:
                continue
            
            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': str(data['desc_id']), 'query': data['desc']})

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': split_type + str(idx), 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'text': "".join(matching_subs)})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': str(data["desc_id"]), 'query': data['desc'], 'docno': split_type + str(idx), 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

def parse_jsonl_QVH(jsonl_path):
    queries_data = []
    documents_data = []
    query_rankings_data = []
    with open(jsonl_path, 'r') as file:
        for idx,line in enumerate(file):

            # Load the JSON object from the line
            data = json.loads(line)

            triple = data["vid"].split("_")
            document_name = triple[0:-2]
            document_name = "".join(document_name)
            start_time = float(triple[-2])
            end_time = float(triple[-1])

            if document_name not in subtitles_dict:
                #print("Document not found in subtitles: ", document_name)
                continue

            momentaneus_rank =[]
            count = 0
            for id,relevant_window in enumerate(data["relevant_windows"]):
                ts = [start_time+relevant_window[0], start_time+relevant_window[1]]
                subs = [sub for sub in subtitles_dict[document_name] if sub[0] <= ts[1] and ts[0] <= sub[1]]
                if len(subs) == 0:
                    #print("No subtitles found for ", document_name, " at time ", ts)
                    continue
                count += 1
                documents_data.append({"docno" : str(idx) +"_"+str(ts[0]) + "_" + str(ts[1]), "vid_name" : document_name, "ts": ts, "duration": data["duration"], "text": "".join([sub[2] for sub in subs])})
                scores = [data["saliency_scores"][i]  for i,clip_id in enumerate(data["relevant_clip_ids"]) if clip_id*2 >= relevant_window[0] and clip_id*2 <= relevant_window[1]]
                if len(scores) == 0:
                    #print("No scores found for ", document_name, " at time ", ts)
                    continue
                #each entry of scores is a triple of integers. Create a variable score which is the average of all the scores
                score = 0 if len(scores) ==0 else sum(sum(scores[i]) for i in range(len(scores)))/(3*len(scores))

                momentaneus_rank.append({"qid" : str(data["qid"]), "query": data["query"] , "docno" : str(idx) +"_"+str(ts[0]) + "_" + str(ts[1]), "score": score, "rank":1})
            
            if count == 0:
                #print("No relevant windows found for ", document_name)
                continue
            #adjust the rank of the momentaneus_rank based on the score
            momentaneus_rank = sorted(momentaneus_rank, key=lambda x: x["score"], reverse=True)
            for i in range(len(momentaneus_rank)):
                momentaneus_rank[i]["rank"] = i+1
            queries_data.append({"qid" : str(data["qid"]), "query": data["query"]})
            query_rankings_data.extend(momentaneus_rank)

    return queries_data, documents_data, query_rankings_data

In [114]:
#CREATION AND SPLIT OF THE DATASETS

if dataset_choice == "TVR":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_TVR(jsonl_train_path, "t") #the letter is just to avoid having the same docno for different documents
    queries_data_val, documents_data_val, query_rankings_data_val = parse_jsonl_TVR(jsonl_val_path, "v")
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]

elif dataset_choice == "QVH":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_QVH(jsonl_train_path)
    #have to create a val set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_val = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_val]
    queries_data_val = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_val]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_val]
    documents_data_val = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_val]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_val]
    #have to create a test set; to do it, extract a random 10% of the remaining train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]



In [ ]:
# Create DataFrames for the Query Set, Documents Set, and Query Rankings
queries_train_df = pd.DataFrame(queries_data_train)
documents_train_df = pd.DataFrame(documents_data_train)
query_rankings_data_train_df = pd.DataFrame(query_rankings_data_train)

queries_val_df = pd.DataFrame(queries_data_val)
documents_val_df = pd.DataFrame(documents_data_val)
query_rankings_data_val_df = pd.DataFrame(query_rankings_data_val)

queries_test_df = pd.DataFrame(queries_data_test)
documents_test_df = pd.DataFrame(documents_data_test)
query_rankings_data_test_df = pd.DataFrame(query_rankings_data_test)

#Sometimes it is useful to have all the qrels data in a single dataframe
q_rels = pd.concat([pd.DataFrame(query_rankings_data_train), pd.DataFrame(query_rankings_data_val), pd.DataFrame(query_rankings_data_test)]).reset_index(drop=True)

#print length of the dataframes
print("Train set:")
print("Queries: ", len(queries_train_df))
print("Documents: ", len(documents_train_df))

print("Val set:")
print("Queries: ", len(queries_val_df))
print("Documents: ", len(documents_val_df))

print("Test set:")
print("Queries: ", len(queries_test_df))
print("Documents: ", len(documents_test_df))

print("Query Rankings: ", len(query_rankings_data_train))
print("Query Rankings: ", len(query_rankings_data_val))
print("Query Rankings: ", len(query_rankings_data_test))

print("Query Rankings: ", len(q_rels))

***INITIALIZATION AND INDEXING:***

In [116]:
if not pt.started():
    pt.init()

In [117]:
# Create an index for the documents

indexer = pt.IterDictIndexer(
    "./index_path/",
    meta={
        "docno": 64,
        "vid_name": 64,
        "text": 131072,
    },
    stemmer="porter",
    stopwords="terrier",
    overwrite=True,
    # type=pt.index.IndexingType.MEMORY,
)

In [ ]:
joint_documents_set_df = pd.concat([documents_train_df, documents_val_df, documents_test_df])

print("Length: ", len(joint_documents_set_df))

In [ ]:
indexed = indexer.index(
    joint_documents_set_df.to_dict(orient="records")
)

#NOTE: when it says 'empy documents added', it is a warning referred to 
#      subtitles only made of stopwords. It is not a problem for the indexing
#      but of course these subtitles will not be useful for the retrieval


***FIRST STAGE RETRIEVERS:***

In [120]:
#These are some examples, not necessarily the best ones. Experiment with different models 

# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=3,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify LemurTF_IDF as the weighting model
first_stage_lemurtfidf = pt.BatchRetrieve(
    indexed,
    wmodel="LemurTF_IDF",
    num_results=3,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify Hiemstra_LM as the weighting model
first_stage_hiemstra_lm = pt.BatchRetrieve(
    indexed,
    wmodel="Hiemstra_LM",
    num_results=3,
    metadata=["docno", "vid_name", "text"]
)

***FSR SCORES AS FEATURES:***

In [121]:
#We create features for the second stage using the first stage retrievers

#TF-IDF based features
lemur_tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="LemurTF_IDF")
bm25_retriever = pt.BatchRetrieve(indexed, wmodel="BM25")
tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="TF_IDF")

#Language model based features
hiem_retriever = pt.BatchRetrieve(indexed, wmodel="Hiemstra_LM")
dirichlet_retriever = pt.BatchRetrieve(indexed, wmodel="DirichletLM")

#Divergence from randomness based features
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2")
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH")
dlh_retriever = pt.BatchRetrieve(indexed, wmodel="DLH")

#Can add more!




In [122]:
# PIPELINES WITH FSR AND FEATURES

#Can test many possibilities, i just put some examples here

bm25_pipeline = ~first_stage_bm25 >> (
   pl2_retriever ** dph_retriever ** tf_idf_retriever  
)


lemurtf_idf_pipeline = ~first_stage_lemurtfidf >> (
    pl2_retriever ** dph_retriever ** hiem_retriever
)

hiem_lm_pipeline = ~hiem_retriever >> (
    pl2_retriever ** dph_retriever ** tf_idf_retriever
)

#NOTE: i believe we should not use the same first stage retriever for features AND for the first stage retrieval


In [123]:
# Prepare the queries for the pipeline, remove special characters and extra spaces

prepared_trainqueries = queries_train_df
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels = query_rankings_data_train_df
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels['label'] = prepared_train_qrels['score']

prepared_train_qrels['label'] = prepared_train_qrels['label'].astype(int)

prepared_val_qrels = query_rankings_data_val_df
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_val_qrels['label'] = prepared_val_qrels['score']

prepared_val_qrels['label'] = prepared_val_qrels['label'].astype(int)

prepared_test_qrels = query_rankings_data_test_df
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_test_qrels['label'] = prepared_test_qrels['score']

prepared_test_qrels['label'] = prepared_test_qrels['label'].astype(int)

prepared_qrels = q_rels
prepared_qrels['query'] = prepared_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_qrels['label'] = prepared_qrels['score']

prepared_qrels['label'] = prepared_qrels['label'].astype(int)

prepared_valqueries = queries_val_df
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

#test set
prepared_testqueries = queries_test_df
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()


***LEARNING TO RANK:***

In [124]:
from sklearn.svm import SVR
import xgboost as xgb
import fastrank


index = pt.IndexFactory.of("./index_path")

#Uncomment the pipelines you want to use

fsr_pipelines = [
    {
        'pipe': bm25_pipeline,
        'name': 'BM25'
    },
    {
        'pipe': lemurtf_idf_pipeline,
        'name': 'LemurTF_IDF'
    },
    # {
    #     'pipe': hiem_lm_pipeline,
    #     'name': 'Hiemstra_LM'
    # }
]

#Uncomment the models you want to use

learned_models = [
    # {
    #     'model': SVR(),
    #     'form': 'reg',
    #     'name': 'SVR',
    # },
    {
        'model': xgb.XGBRanker(tree_method="hist", objective="rank:ndcg"),
        'form': 'ltr',
        'name': 'XGBoost (NDCG)',
     },
    # {
    #     'model': xgb.XGBRanker(tree_method="hist", objective="rank:map"),
    #     'form': 'ltr',
    #     'name': 'XGBoost (MAP)',
    # },
#     {
#         'model': fastrank.TrainRequest.coordinate_ascent(),
#         'form': 'fastrank',
#         'name': 'FastRank Coordinate Ascent',
#     },
#    {
#         'model': fastrank.TrainRequest.random_forest(),
#         'form': 'fastrank',
#         'name': 'FastRank Random Forest',
#     },
]

***TRAINING:***

In [ ]:
#Put the names of the first stage retrievers you are using in the list

trained_models = [first_stage_bm25, first_stage_lemurtfidf]
names = ['BM25','LemurTF_IDF']

for fsr in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr['name']} >> {model['name']}")
        print(names[-1])
        pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
        pipe.fit(
            prepared_trainqueries,
            prepared_train_qrels,
            prepared_valqueries,
            prepared_val_qrels
        )
        trained_models.append(pipe)

***EXPERIMENTS AND RESULTS:***

In [ ]:
#Results on the test set

#NOTE: These are the only ones that really matter, but it could be useful 
#      to compare them with the ones on the train/val set to see if there is overfitting

from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_testqueries,
    prepared_test_qrels,
    names=names,
    eval_metrics=[nDCG @ 1, RR @ 1, nDCG @ 3, RR @ 3, nDCG @ 5, RR @ 5, nDCG @ 10, RR @ 10, MAP],
)

In [ ]:
#Results on the train set

pt.Experiment(
    trained_models,
    prepared_trainqueries,
    prepared_train_qrels,
    names=names,
    eval_metrics=[nDCG @ 1, RR @ 1, nDCG @ 3, RR @ 3, nDCG @ 5, RR @ 5, nDCG @ 10, RR @ 10, MAP],
)

In [ ]:
#Results on the val set

pt.Experiment(
    trained_models,
    prepared_valqueries,
    prepared_val_qrels,
    names=names,
    eval_metrics=[nDCG @ 1, RR @ 1, nDCG @ 3, RR @ 3, nDCG @ 5, RR @ 5, nDCG @ 10, RR @ 10, MAP],
)